In [4]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
#from tensorflow.keras.applications import SqueezeNet
from tensorflow.keras.optimizers import Adam

In [2]:
pip install keras_applications

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 390.5 kB/s eta 0:00:00


In [8]:
from google.colab import files

# Upload the .zip file
uploaded = files.upload()

# Extract the contents of the .zip file
import zipfile
import io

with zipfile.ZipFile("/content/brain_tumor_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')
    zip_ref.close()

Saving brain_tumor_dataset.zip to brain_tumor_dataset.zip


In [9]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os

In [10]:
data_dir = '/content/dataset/brain_tumor_dataset'
os.listdir(data_dir)

['yes', 'no']

In [11]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

Found 253 files belonging to 2 classes.


In [12]:
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

In [13]:
data = data.map(lambda x,y: (x/255, y))
scaled_iterator=data.as_numpy_iterator().next()

In [14]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.15)
test_size = int(len(data)*.15)

In [15]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Dropout, Flatten, Dense
from tensorflow.keras.models import Model

def fire_module(x, squeeze, expand):
    squeezed = Conv2D(squeeze, (1, 1), activation='relu', padding='same')(x)
    expanded1x1 = Conv2D(expand, (1, 1), activation='relu', padding='same')(squeezed)
    expanded3x3 = Conv2D(expand, (3, 3), activation='relu', padding='same')(squeezed)
    output = concatenate([expanded1x1, expanded3x3], axis=-1)
    return output

def SqueezeNet(input_shape=(224, 224, 3), num_classes=1000):
    input_img = Input(shape=input_shape)
    x = Conv2D(96, (7, 7), strides=(2, 2), activation='relu', padding='same')(input_img)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = fire_module(x, squeeze=16, expand=64)
    x = fire_module(x, squeeze=16, expand=64)
    x = fire_module(x, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = fire_module(x, squeeze=32, expand=128)
    x = fire_module(x, squeeze=48, expand=192)
    x = fire_module(x, squeeze=48, expand=192)
    x = fire_module(x, squeeze=64, expand=256)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = fire_module(x, squeeze=64, expand=256)
    x = Dropout(0.5)(x)

    x = Conv2D(num_classes, (1, 1), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_img, outputs=output)
    return model

num_classes = 100

# Load SqueezeNet
transfer_model = SqueezeNet(input_shape=(224, 224, 3), num_classes=num_classes)

transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

transfer_model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_26 (Conv2D)          (None, 112, 112, 96)         14208     ['input_2[0][0]']             
                                                                                                  
 max_pooling2d_3 (MaxPoolin  (None, 56, 56, 96)           0         ['conv2d_26[0][0]']           
 g2D)                                                                                             
                                                                                                  
 conv2d_27 (Conv2D)          (None, 56, 56, 16)           1552      ['max_pooling2d_3[0][0]'

In [19]:
#hist = transfer_model.fit(train, epochs=30, validation_data=val)